In [17]:
# 2m : 100m
# height: 0.02m : 1 -> 10m : 500 -10m : -500 
# 10m : 1

from PIL import Image, ImageDraw
import numpy
import math

import json
    
total_distance = 3200
race_track = "Kyoto_outside_double"
# direction = "right"

with open(race_track + '.json') as track_file:
    track_data = json.load(track_file)

# print(track_data)

slope_interval = {}
pos_arr = []
height_arr = []
slope_arr = []
speed_minus_per = []
for key in track_data.keys():
    pos_arr.append(int(key))
    height_arr.append(track_data[key])
    
for i in range(0, len(pos_arr) - 1):
    slope_arr.append((height_arr[i + 1] - height_arr[i]) / (pos_arr[i + 1] - pos_arr[i]))
    
for i in range(0, len(slope_arr)):
    if slope_arr[i] <= 0.01:
        speed_minus_per.append(0)
    else:
        speed_minus_per.append(int(slope_arr[i] * 200 * 10000) / 10000)
# print(slope_arr)

#15.980 
base_speed_first = 18.756 
base_speed_middle = 18.381 
base_speed_last = 22.457

first_part = 320000 / 6
middle_part = 320000 / 2
last_part = 320000 / 3

for k in range(1, 12):
    power = k
    acc = 0.0006 * math.sqrt(50000 * power)
#     print(acc)
#     print((base_speed_middle - base_speed_first) / acc * (-1))

    speed_array = []
    speed_array_no_saka = []

    for i in range(0, len(pos_arr) - 1):
        if pos_arr[i] < first_part:
            speed_array.append(base_speed_first - speed_minus_per[i] / power)
            speed_array_no_saka.append(base_speed_first)

        elif pos_arr[i] < middle_part:
            speed_array.append(base_speed_middle - speed_minus_per[i] / power)
            speed_array_no_saka.append(base_speed_middle)
        else:
            speed_array.append(base_speed_last - speed_minus_per[i] / power)
            speed_array_no_saka.append(base_speed_last)








    total_time = 0
    total_time_no_saka = 0
    for i in range(1, len(pos_arr) - 1):
        total_time += (pos_arr[i] - pos_arr[i - 1]) / speed_array[i]
        total_time_no_saka += (pos_arr[i] - pos_arr[i - 1]) / speed_array_no_saka[i]
    #     print(total_time)


    total_time_with_acc = total_time + 53.3 + 797.4 + 46.9 + (base_speed_last - base_speed_middle) * 100 / acc
    total_time_with_acc_no_saka = total_time_no_saka + 53.3 + 797.4 + 46.9 + (base_speed_last - base_speed_middle) * 100 / acc
    # print("", total_time, total_time_with_acc)
    # print("", total_time_no_saka, total_time_with_acc_no_saka)
    # print(total_time_with_acc * 1.18)
    print("", k * 100)
    print((total_time + total_time_with_acc) * 0.59)
    print((total_time_no_saka + total_time_with_acc_no_saka) * 0.59)
    # print(speed_minus_per)

 100
21583.62530156498
20896.868541307846
 200
20656.744680388754
20371.86860435087
 300
20319.44698251493
20139.284999670253
 400
20132.41796258295
20000.637588806083
 500
20009.912364492615
19906.01996963765
 600
19921.925465873137
19836.176144743607
 700
19854.895663080486
19781.89355561809
 800
19801.692492649134
19738.13762032759
 900
19758.166825199874
19701.89393797216
 1000
19721.72108373897
19671.232860193868
 1100
19690.63631544336
19644.854235386352


In [1]:
# 2m : 100m
# height: 0.02m : 1 -> 10m : 500 -10m : -500 
# 10m : 1

# speed 1.0000 0.98 0.962 0.962
# acc 1.0000 1 0.996 0.996
# tension 1.04 1.02 1 0.98 0.96

from PIL import Image, ImageDraw
import numpy
import math

import json
    
tension_bonus = {
    'perfect': 1.04,
    'great': 1.02,
    'normal': 1,
    'bad': 0.98,
    'terrible': 0.96
}

course_bonus = {
    'acc':{
        'B': 0.9,
        'A': 1,
        'S': 1.05
    }
}

distance_bonus = {
    'speed':{
        'B': 0.9,
        'A': 1,
        'S': 1.05
    },
    'power':{
        'B': 1,
        'A': 1,
        'S': 1
    }
}

tatic_bonus = {
    'smart':{
        'B': 0.85,
        'A': 1,
        'S': 1.1
    }
}

tatic_param ={
    'nige':{
        'speed':{
            'first': 1, 
            'middle': 0.98, 
            'last': 0.962,
            'final_part': 0.962
        },
        'acc':{
            'first': 1, 
            'middle': 1, 
            'last': 0.996,
            'final_part': 0.996
        }
    },
    'senkou':{
        'speed':{
            'first': 0.978, 
            'middle': 0.991, 
            'last': 0.975,
            'final_part': 0.975
        },
        'acc':{
            'first': 0.985, 
            'middle': 1, 
            'last': 0.996,
            'final_part': 0.996
        }
    },
    'sashi':{
        'speed':{
            'first': 0.938, 
            'middle': 0.998, 
            'last': 0.994,
            'final_part': 0.994
        },
        'acc':{
            'first': 0.975, 
            'middle': 1, 
            'last': 1,
            'final_part': 1
        }
    }, 
    'oikomu':{
        'speed':{
            'first': 0.931, 
            'middle': 1, 
            'last': 1,
            'final_part': 1
        },
        'acc':{
            'first': 0.945, 
            'middle': 1, 
            'last': 0.997,
            'final_part': 0.997
        }
    }

}

    
    
    
    
    
def cal_uma_real_status(uma):
    
    attr_ori = uma['attr_ori']
    aptitude = uma['aptitude']
    uma['attr'] = {
        'speed': attr_ori['speed'] * tension_bonus[uma['tension']],
        'power': attr_ori['power'] * tension_bonus[uma['tension']],
        'smart': attr_ori['smart'] * tension_bonus[uma['tension']] * tatic_bonus['smart'][aptitude['tactic']]
    }
    
    acc_temp = math.sqrt(uma['attr']['power'] * 500) * course_bonus['acc'][aptitude['course']]
    acc_normal = acc_temp * 0.0006
    acc_saka = acc_temp * 0.0004
    uma['acc'] = {
        'first': {
            'normal': acc_normal * tatic_param[uma['tatic']]['acc']['first'],
            'saka': acc_saka * tatic_param[uma['tatic']]['acc']['first'],
            'minus': -1.2
        }, 
        'middle': {
            'normal': acc_normal * tatic_param[uma['tatic']]['acc']['middle'],
            'saka': acc_saka * tatic_param[uma['tatic']]['acc']['middle'],
            'minus': -0.8
        }, 
        'last': {
            'normal': acc_normal * tatic_param[uma['tatic']]['acc']['last'],
            'saka': acc_saka * tatic_param[uma['tatic']]['acc']['last'],
            'minus': -1
        },
        'final_part': {
            'normal': acc_normal * tatic_param[uma['tatic']]['acc']['final_part'],
            'saka': acc_saka * tatic_param[uma['tatic']]['acc']['final_part'],
            'minus': -1,
            'exhausted': -1.2
        }
    }
    
    return uma
    
def cal_uma_base_speed(uma, track):
    
    track_base_speed = track["base_speed"]
    
    smart = uma["attr"]["smart"]
    speed_variation_by_smart_top = (smart / 5500) * math.log(smart * 0.1, 10)
    speed_variation_by_smart_bottom  = speed_variation_by_smart_top - 0.65
    speed_variation_by_smart_avg = speed_variation_by_smart_top - 0.325
    
    speed_variation_by_smart_avg = speed_variation_by_smart_avg * 0.01
    
    
    uma_base_speed = {
        'first': track_base_speed * (tatic_param[uma['tatic']]['speed']['first'] + speed_variation_by_smart_avg), 
        'middle': track_base_speed * (tatic_param[uma['tatic']]['speed']['middle'] + speed_variation_by_smart_avg), 
        'last': track_base_speed * (tatic_param[uma['tatic']]['speed']['last'] + speed_variation_by_smart_avg),
#         'final_part': track_base_speed * (tatic_param[uma['tatic']]['speed']['final_part'] + speed_variation_by_smart_avg)
    }
    
    speed_constant = math.sqrt(uma['attr']['speed'] * 500) * distance_bonus['speed'][uma['aptitude']['distance']] * 0.002
    uma_base_speed['last'] += speed_constant
    uma_base_speed['final_part'] = (uma_base_speed['last'] + 0.01 * track_base_speed) * 1.05
    uma_base_speed['final_part'] += speed_constant
    
    return uma_base_speed
    
    
    
def cal_before_race(uma, track):
    
    track["base_speed"] = 22 - track["distance"] / 1000
    uma_base_speed = cal_uma_base_speed(uma, track)

    print(uma['attr_ori']['speed'], uma_base_speed)
    
    
for k in range(16, 30):
    speed = k * 50
    uma = {
        "attr_ori":{
            'speed': speed,
            'power': 600,
            'smart': 300
        },
        'aptitude':{
            'course': 'A',
            'distance': 'S',
            'tactic': 'A'
        },
        'tension': 'perfect',
        'tatic': 'nige'
    }

    track = {
        'name': "Kyoto_outside_double",
        'distance': 3200
    }



    uma = cal_uma_real_status(uma)
    cal_before_race(uma, track)

800 {'first': 18.754834751248502, 'middle': 18.3788347512485, 'last': 19.394894052962655, 'final_part': 21.916498057324944}
850 {'first': 18.754834751248502, 'middle': 18.3788347512485, 'last': 19.43657944289045, 'final_part': 22.001953106676925}
900 {'first': 18.754834751248502, 'middle': 18.3788347512485, 'last': 19.477055786873414, 'final_part': 22.084929611841996}
950 {'first': 18.754834751248502, 'middle': 18.3788347512485, 'last': 19.51642255607617, 'final_part': 22.165631488707643}
1000 {'first': 18.754834751248502, 'middle': 18.3788347512485, 'last': 19.554766286943376, 'final_part': 22.24423613698542}
1050 {'first': 18.754834751248502, 'middle': 18.3788347512485, 'last': 19.59216282020904, 'final_part': 22.32089903018003}
1100 {'first': 18.754834751248502, 'middle': 18.3788347512485, 'last': 19.62867906494838, 'final_part': 22.395757331895677}
1150 {'first': 18.754834751248502, 'middle': 18.3788347512485, 'last': 19.66437440529972, 'final_part': 22.468932779615926}
1200 {'firs

In [3]:
from PIL import Image, ImageDraw
import numpy
import math

import json


def track_divide(track):
    with open(track['name'] + '.json') as track_file:
        track_data = json.load(track_file)
    
    slope_dict = {}
    
    pos_array = []
    height_array = []
    
    for key in track_data.keys():
        pos_array.append(int(key))
        height_array.append(track_data[key])
        
    slope_array = []
    for i in range(0, len(pos_array) - 1):
        
        slope = (height_array[i + 1] - height_array[i]) / (pos_array[i + 1] - pos_array[i])
        if slope >= 0.007:
            slope_array.append('uphill')
        elif slope <= -0.007:
            slope_array.append('downhill')
        else:
            slope_array.append('')
    
    for i in range(1, len(slope_array) - 1):
        if slope_array[i - 1] == slope_array[i + 1]:
            if slope_array[i] != slope_array[i + 1]:
                slope_array[i] = slope_array[i + 1]
                
    for i in range(1, len(slope_array) - 2):
        if slope_array[i] != '':
            if slope_array[i - 1] != slope_array[i] and slope_array[i + 1] != slope_array[i]:
                slope_array[i] = ''
                
    for i in range(0, len(slope_array)):
        if slope_array[i] != '':
            slope = (height_array[i + 1] - height_array[i]) / (pos_array[i + 1] - pos_array[i])
            slope_dict[pos_array[i]] = {
                'height':height_array[i],
                'slope': slope,
                'type': slope_array[i]
            }
        else:
            slope_dict[pos_array[i]] = {
                'height':height_array[i],
                'slope': 0.0,
                'type': ''
            }            

    return slope_dict



def cal_startdash(uma, track):
    
    
    speed_end = track["base_speed"] * 0.85
    startdash_dict = {
        'cost_time': (speed_end - 3) / (24 + uma['acc']['first']['normal'])
    }
    startdash_dict['distance'] = (3 + speed_end) * startdash_dict['cost_time'] / 2
    
    return startdash_dict


    
def cal_slope_interval(uma, track, slope_dict):
    
    slope_interval = []
    start = -1
    end = -1
    slope_tmp = 0.0
    pos_array = []
    slope_array = []
    height_array = []
    for pos in slope_dict.keys():
        pos_array.append(pos)
        height_array.append(slope_dict[pos]['height'])
        slope_array.append(slope_dict[pos]['slope'])
        
    for i in range(1, len(pos_array) -2):
        if slope_array[i] != 0.0:
            if abs(slope_array[i - 1] - slope_array[i + 1]) < 0.001:
                slope_array[i] = (slope_array[i - 1] + slope_array[i + 1]) / 2
            
    i = 0
    while i < len(pos_array):
        if slope_dict[pos_array[i]]['type'] == "":
            i += 1
        else:
            j = 1
            end = 0
            while j + i < len(pos_array) - 1:
                if abs(slope_array[i + j] - slope_array[i]) < 0.001:
                    end += 1
                    j += 1
                else:
                    if j != 1:
                        slope_interval.append({
                            'start': pos_array[i],
                            'end': pos_array[i + j],
                            'slope': (height_array[i + j] - height_array[i]) / (pos_array[i + j] - pos_array[i])
                        })
                    break
            i += j
    return slope_interval

def cal_track_interval(track, slope_interval):
    
#     print(slope_interval)
#     print(track)

    race_interval = []
    tmp = []
    for i in track['interval'].keys():
        tmp.append(int(track['interval'][i]['start'] * 100))
        tmp.append(int(track['interval'][i]['end'] * 100))
        
    for i in slope_interval:
        tmp.append(i['start'])
        tmp.append(i['end'])
    tmp.sort()
    
    tmp2 = []
    for i in tmp:
        if i in tmp2:
            continue
        else:
            tmp2.append(i)
                
    for i in range(0, len(tmp2) - 1):
        race_interval.append({
            'start': tmp2[i],
            'end': tmp2[i + 1],
            'distance': tmp2[i + 1] - tmp2[i],
            'part': '',
            'slope': 0.0
        })
        
    for i in track['interval'].keys():
        for j in race_interval:
            if j['start'] >= int(track['interval'][i]['start'] * 100) and j['end'] <= int(track['interval'][i]['end'] * 100):
                j['part'] = i
    
    for i in slope_interval:
        for j in race_interval:
            if j['start'] >= i['start'] and j['end'] <= i['end']:
                j['slope'] = i['slope']
    
    return race_interval
            
def cal_race(uma, track):
    
    track["base_speed"] = 22 - track["distance"] / 1000
    race_result = {}
    
    uma["base_speed"] = cal_uma_base_speed(uma, track)
    slope_dict = track_divide(track)
    slope_interval = cal_slope_interval(uma, track, slope_dict)
    race_interval = cal_track_interval(track, slope_interval)
    
#     print(race_interval)
    for i in race_interval:
        if i['part'] == 'last':
            part = 'final_part'
        else:
            part = i['part']
        
        if i['slope'] >= 0.02:
            i['speed'] = uma['base_speed'][part] - i['slope'] * 20000 / uma['attr']['power']
            i['acc'] =  uma['acc'][part]
        elif i['slope'] <= -0.02:
            i['speed'] = uma['base_speed'][part]
            i['acc'] =  uma['acc'][part]
        else:
            i['speed'] = uma['base_speed'][part]
            i['acc'] =  uma['acc'][part]
        print(i['slope'])

#         print(uma['base_speed'][i['part']], uma['base_speed'][i['part']] - i['slope'] * 20000 / uma['attr']['power'])
        
    current_speed = track["base_speed"] * 0.85
    current_time = 0
    
    for i in race_interval:
        cost_time = 0
        remain_distance = i['distance']
        
        if current_speed < i['speed']:
            if i['slope'] >= 0.02:
                acc = i['acc']['saka']
                acc_time = (i['speed'] - current_speed) / acc
                acc_distance = current_speed * acc_time + 0.5 * acc * acc_time * acc_time
                
                remain_distance = remain_distance - acc_distance
                keep_time = remain_distance / i['speed']
                
                cost_time = acc_time + keep_time 
            else:
                acc = i['acc']['normal']
                acc_time = (i['speed'] - current_speed) / acc
                acc_distance = current_speed * acc_time + 0.5 * acc * acc_time * acc_time
                
                remain_distance = remain_distance - acc_distance
                keep_time = remain_distance / i['speed']
                
                cost_time = acc_time + keep_time
        elif current_speed == i['speed']:
            keep_time = remain_distance / i['speed']
            cost_time = keep_time 
        else:
            acc = i['acc']['minus']
            acc_time = (i['speed'] - current_speed) / acc
            acc_distance = current_speed * acc_time + 0.5 * acc * acc_time * acc_time
            
            remain_distance = remain_distance - acc_distance
            keep_time = remain_distance / i['speed']

            cost_time = acc_time + keep_time
        
        current_speed = i['speed']
        current_time += cost_time
#         print(current_time)
        
    print(uma['attr_ori']['speed'], uma['attr_ori']['power'], current_time * 1.18)
        
    
    
    
#     race_result["startdash"] = cal_startdash(uma, track)
    
#     cal_speed_array(uma, track, slope_dict)
    

# for k in range(6, 30):

# for i in range(10, 20):
print()
total = i * 100
for j in range(5, 20):
    uma = {
        "attr_ori":{
            'speed': total - j * 50,
            'power': 900,
            'smart': 329
        },
        'aptitude':{
            'course': 'A',
            'distance': 'A',
            'tactic': 'A'
        },
        'tension': 'perfect',
        'tatic': 'nige'
    }

    track = {
        'name': "Kyoto_outside_double",
        'distance': 3200,
        'interval': {}
    }

    track['interval']['first'] = {
        'start': 0,
        'end': track['distance'] / 6,
        'distance': track['distance'] / 6
    }

    track['interval']['middle'] = {
        'start': track['interval']['first']['end'],
        'end': track['interval']['first']['end'] + track['distance'] / 2,
        'distance': track['distance'] / 2
    }

    track['interval']['last'] = {
        'start': track['interval']['middle']['end'],
        'end': track['distance'],
        'distance': track['distance'] - track['interval']['middle']['end']
    }

    uma = cal_uma_real_status(uma)
    cal_race(uma, track)
#         break
#     print(uma)
#     break
#     break


0.0
0.022675736961451247
0.0
-0.02415591545429591
-0.02415591545429591
0.0
-0.0245474071801166
0.0
0.022675736961451247
0.0
0.0
-0.02415591545429591
0.0
-0.0245474071801166
0.0
1650 900 19145.578201543824
0.0
0.022675736961451247
0.0
-0.02415591545429591
-0.02415591545429591
0.0
-0.0245474071801166
0.0
0.022675736961451247
0.0
0.0
-0.02415591545429591
0.0
-0.0245474071801166
0.0
1600 900 19159.452597749703
0.0
0.022675736961451247
0.0
-0.02415591545429591
-0.02415591545429591
0.0
-0.0245474071801166
0.0
0.022675736961451247
0.0
0.0
-0.02415591545429591
0.0
-0.0245474071801166
0.0
1550 900 19173.61813792769
0.0
0.022675736961451247
0.0
-0.02415591545429591
-0.02415591545429591
0.0
-0.0245474071801166
0.0
0.022675736961451247
0.0
0.0
-0.02415591545429591
0.0
-0.0245474071801166
0.0
1500 900 19188.089610906874
0.0
0.022675736961451247
0.0
-0.02415591545429591
-0.02415591545429591
0.0
-0.0245474071801166
0.0
0.022675736961451247
0.0
0.0
-0.02415591545429591
0.0
-0.0245474071801166
0.0
145

In [94]:
nige a a a
1200 1000 3.15.6
1200 900 3.15.7
1200 800 3.16.0
1200 700 3.16.3
1200 600 3.16.6
1200 500 3.17.1

1100 1000 3.15.9
1100 900 3.16.0
1100 800 3.16.2
1100 700 3.16.5
1100 600 3.16.8
1100 500 3.17.4

1000 1000 3.16.2
1000 900 3.16.3
1000 800 3.16.5
1000 700 3.16.7
1000 600 3.17.1
1000 500 3.17.6

900 1000 3.16.4
900 900 3.16.6
900 800 3.16.8
900 700 3.17.0
900 600 3.17.3
900 500 3.17.9



oikomu a a a
1200 1200 3.13.9
1200 1100 3.14.1
1200 1000 3.14.2
1200 900 3.14.4
1200 800 3.14.6
1200 700 3.14.9
1200 600 3.15.4


1100 1200 3.14.2
1100 1100 3.14.3
1100 1000 3.14.5
1100 900 3.14.7
1100 800 3.14.9
1100 700 3.15.1
1100 600 3.15.6

1000 1200 3.14.5
1000 1100 3.14.6
1000 1000 3.14.8
1000 900 3.15.0
1000 800 3.15.1
1000 700 3.15.4
1000 600 3.15.9

900 1200 3.14.8
900 1100 3.14.9
900 1000 3.15.0
900 900 3.15.2
900 800 3.15.4
900 700 3.15.7
900 600 3.16.1

SyntaxError: invalid syntax (<ipython-input-94-464c30582213>, line 1)